## merge etf ownership with stock level data

In [73]:
import pandas as pd

############### import formatted_final
ownership_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/fund_holdings_data/formatted_final.csv"
df_ownership = pd.read_csv(ownership_path)

############### import m_stock_level_data
path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/m_stock_level_data.csv"
df_add = pd.read_csv(path)

###### lag df_add by one month
df_add['date'] = pd.to_datetime(df_add['date'])
#df_add['date'] = df_add['date'] + pd.DateOffset(months=1)

##### change df_ownership also to date format
df_ownership["date"] = pd.to_datetime(df_ownership['date'])

result_df = pd.merge(df_ownership, df_add[['date', 'stock_RIC', 'price', "return1Mo", "gross_profit", "price_to_BV"]], on=['date', 'stock_RIC'], how='left')

display(result_df)
output_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv"
result_df.to_csv(output_path, index=False)

,date,stock_RIC,index_member,stock_value_held,FUND_stock_value_held,market_cap,ETF_ownership,FUND_ownership,price,return1Mo,gross_profit,price_to_BV
0,2009-12-31,0MW4EUR.xbo^K15,0,14.731695,9.666637,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-31,0MW4EUR.xbo^K15,0,11.226477,9.598608,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-02-28,0MW4EUR.xbo^K15,0,9.887296,8.259427,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-03-31,0MW4EUR.xbo^K15,0,11.585353,9.845493,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-04-30,0MW4EUR.xbo^K15,0,15.047999,9.688948,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
196219,2023-07-31,ZURN.S,1,5870.052977,3874.537377,6.606391e+10,0.088854,0.058648,439.079073,-0.964933,NaN,2.772687
196220,2023-08-31,ZURN.S,1,5816.278767,3834.866671,6.344198e+10,0.091679,0.060447,433.477862,-1.378327,NaN,2.907803
196221,2023-09-30,ZURN.S,1,5789.045875,3802.279416,6.357877e+10,0.091053,0.059804,434.412512,1.669894,NaN,2.841227
196222,2023-10-31,ZURN.S,1,6022.326667,3973.467134,6.545329e+10,0.092010,0.060707,447.220479,2.499405,NaN,2.926635


## dependent variable: daily volatility for each month

In [74]:
df_dependent = pd.read_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv")
df_dependent['date'] = pd.to_datetime(df_dependent['date'], errors='coerce')
df_dependent.dropna(subset=['date'], inplace=True)
df_dependent.dropna(subset=['return1D'], inplace=True)

# Group by 'stock_RIC' and month, then calculate standard deviation and count of 'returns'
monthly_stats = df_dependent.groupby(['stock_RIC', pd.Grouper(key='date', freq='M')])['return1D'].agg(['std', 'count']).reset_index()
monthly_stats.rename(columns={'std': 'monthly_std_dev', 'count': 'count_returns'}, inplace=True)

# Create a year-month identifier for merging
df_dependent['year_month'] = df_dependent['date'].dt.to_period('M')
monthly_stats['year_month'] = monthly_stats['date'].dt.to_period('M')

# Merge the calculated std dev and count back onto the original DataFrame
df_dependent = pd.merge(df_dependent, monthly_stats, on=['stock_RIC', 'year_month'], how='left')

#Ensure columns are sorted before dropping duplicates
if 'date_x' in df_dependent.columns and 'stock_RIC' in df_dependent.columns:
    df_dependent.sort_values(by=['stock_RIC', 'date_x'], ascending=[True, False], inplace=True)
    df_dependent = df_dependent.drop_duplicates(subset=['stock_RIC', 'year_month'], keep='first')
else:
    print("Error: Check your column names and data integrity.")

# Drop the helper 'year_month' column if no longer needed
df_dependent.drop('year_month', axis=1, inplace=True)
df_dependent.drop('index', axis=1, inplace=True)

df_dependent['date_y'] = df_dependent['date_y'].dt.date
df_dependent['date_y'] = pd.to_datetime(df_dependent['date_y'])
df_dependent.rename(columns={'date_y': 'DATE_monthly_std_dev'}, inplace=True)

df_dependent.rename(columns={'date_x': 'date'}, inplace=True)
# Display the updated DataFrame

df_dependent['date'] = df_dependent["date"] - pd.offsets.MonthEnd(1)

display(df_dependent)
df_dependent.to_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/monthly_st_dev.csv")

,stock_RIC,date,price,return1D,return1Wk,return1Mo,volume,turnover,market_cap,gross_profit,price_to_BV,bid_price,ask_price,DATE_monthly_std_dev,monthly_std_dev,count_returns
1185542,0MW4EUR.xbo^K15,2015-10-31,14.970000,-0.133422,0.133779,-0.066756,938065.0,1.404420e+07,7.302660e+09,2.235776e+09,NaN,14.760000,15.190000,2015-11-30,0.008801,2076
1165736,0MW4EUR.xbo^K15,2015-09-30,14.950000,-0.066845,-0.333333,-0.133601,4538663.0,6.787420e+07,7.291920e+09,2.235776e+09,NaN,14.650000,15.260000,2015-10-31,0.149910,22
1145989,0MW4EUR.xbo^K15,2015-08-31,14.970000,0.066845,-0.200000,0.335121,7431477.0,1.112457e+08,7.305110e+09,2.235776e+09,NaN,14.930000,14.980000,2015-09-30,0.201650,22
1127377,0MW4EUR.xbo^K15,2015-07-31,14.940000,0.134048,0.403226,-1.059603,3412482.0,5.096370e+07,7.290470e+09,2.235776e+09,NaN,14.870000,15.000000,2015-08-31,0.498490,21
1106891,0MW4EUR.xbo^K15,2015-06-30,15.100000,0.599600,-0.132275,-0.264201,1315684.0,1.978780e+07,7.367691e+09,2.235776e+09,NaN,14.660000,15.410000,2015-07-31,0.483467,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78981,ZURN.S,2010-04-30,166.645847,0.042248,3.001305,-1.415487,286368.0,4.778093e+07,2.468892e+10,NaN,1.053197,166.645847,166.786595,2010-05-31,2.251636,19
63612,ZURN.S,2010-03-31,167.705219,-1.678264,-3.417772,-5.938007,1179841.0,1.999244e+08,2.474560e+10,NaN,1.100166,167.705219,167.844857,2010-04-30,1.236923,20
48149,ZURN.S,2010-02-28,189.913011,-0.331858,1.235955,4.362934,1636985.0,3.115723e+08,2.802245e+10,NaN,1.265999,189.842751,189.913011,2010-03-31,0.622809,23
30553,ZURN.S,2010-01-31,177.091914,1.092896,1.688261,13.199301,876093.0,1.546427e+08,2.612145e+10,NaN,1.219970,177.023538,177.091914,2010-02-28,1.339416,20


#### GPT code

In [50]:
import pandas as pd

# Load the data
df_dependent = pd.read_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv")
df_dependent['date'] = pd.to_datetime(df_dependent['date'], errors='coerce')
df_dependent.dropna(subset=['date', 'return1D'], inplace=True)

# Adjust date to the first day of each month right after loading the data
df_dependent['date'] = df_dependent['date'].dt.to_period('M').dt.start_time

# Group by 'stock_RIC' and month, then calculate standard deviation and count of 'returns'
monthly_stats = df_dependent.groupby(['stock_RIC', 'date']).agg(
    monthly_std_dev=('return1D', 'std'),
    count_returns=('return1D', 'count')
).reset_index()

# Ensure date is the first day of the month (redundant safety if previous did not apply correctly)
monthly_stats['date'] = monthly_stats['date'].dt.to_period('M').dt.start_time

# Merge the calculated std dev and count back onto the original DataFrame
df_dependent = pd.merge(df_dependent, monthly_stats, on=['stock_RIC', 'date'], how='left')

# Ensure columns are sorted before dropping duplicates
df_dependent.sort_values(by=['stock_RIC', 'date'], ascending=[True, False], inplace=True)
df_dependent.drop_duplicates(subset=['stock_RIC', 'date'], keep='first', inplace=True)


# Display the updated DataFrame
display(df_dependent)

,index,stock_RIC,date,price,return1D,return1Wk,return1Mo,volume,turnover,market_cap,gross_profit,price_to_BV,bid_price,ask_price,monthly_std_dev,count_returns
1185541,1185670,0MW4EUR.xbo^K15,2015-11-01,14.990000,0.267559,-1.332445e-01,0.066756,1368739.0,2.050580e+07,7.314502e+09,2.235776e+09,NaN,14.680000,15.290000,0.008801,2076
1165715,1165837,0MW4EUR.xbo^K15,2015-10-01,14.980000,0.066800,3.478110e-10,0.334896,9093735.0,1.360759e+08,7.309623e+09,2.235776e+09,NaN,14.660000,15.270000,0.149910,22
1145968,1146090,0MW4EUR.xbo^K15,2015-09-01,14.930000,-0.066934,1.611380e-10,-1.125828,4251369.0,6.351520e+07,7.286203e+09,2.235776e+09,NaN,14.910000,15.050000,0.201650,22
1127357,1127479,0MW4EUR.xbo^K15,2015-08-01,15.120000,0.132450,9.345794e-01,-0.657030,2085480.0,3.154820e+07,7.377329e+09,2.235776e+09,NaN,14.810000,15.560000,0.498490,21
1106869,1106990,0MW4EUR.xbo^K15,2015-07-01,15.230000,0.594452,-8.463542e-01,-1.805287,4092127.0,6.234840e+07,7.429293e+09,2.235776e+09,NaN,14.440000,16.040000,0.483467,23
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78963,78965,ZURN.S,2010-05-01,166.622150,-0.666112,-4.483587e+00,-7.519380,767875.0,1.281695e+08,2.458578e+10,NaN,1.084582,166.622150,166.691983,2.251636,19
63593,63594,ZURN.S,2010-04-01,180.289039,1.368849,2.775694e+00,4.860314,1716863.0,3.092353e+08,2.660239e+10,NaN,1.208161,180.219159,180.289039,1.236923,20
48127,48128,ZURN.S,2010-03-01,178.573161,0.888031,2.310102e+00,12.387097,824560.0,1.466615e+08,2.633994e+10,NaN,1.224302,178.573161,178.641501,0.622809,23
30534,30534,ZURN.S,2010-02-01,158.148896,2.152900,3.379280e+00,2.649007,822222.0,1.292777e+08,2.331998e+10,NaN,1.113613,158.148896,158.216917,1.339416,20


#### merge dependent variable to monthly_panel

In [75]:
# Assume df_dependent is defined earlier in your script
df_dependent_sub = df_dependent[["stock_RIC", "date", "DATE_monthly_std_dev", "monthly_std_dev", "count_returns"]].copy()

display(df_dependent_sub)

# # Save the result to CSV
df_dependent_sub.to_csv("/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/monthly_st_dev.csv", index=False)

output_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv"
df_monthly = pd.read_csv(output_path)
df_monthly["date"] = pd.to_datetime(df_monthly['date'])

# Merge DataFrames
merged_df = pd.merge(df_monthly, df_dependent_sub, on=['date', 'stock_RIC'], how='left')
display(merged_df)

# Save the merged DataFrame
merged_df.to_csv(output_path, index = False)

,stock_RIC,date,DATE_monthly_std_dev,monthly_std_dev,count_returns
1185542,0MW4EUR.xbo^K15,2015-10-31,2015-11-30,0.008801,2076
1165736,0MW4EUR.xbo^K15,2015-09-30,2015-10-31,0.149910,22
1145989,0MW4EUR.xbo^K15,2015-08-31,2015-09-30,0.201650,22
1127377,0MW4EUR.xbo^K15,2015-07-31,2015-08-31,0.498490,21
1106891,0MW4EUR.xbo^K15,2015-06-30,2015-07-31,0.483467,23
...,...,...,...,...,...
78981,ZURN.S,2010-04-30,2010-05-31,2.251636,19
63612,ZURN.S,2010-03-31,2010-04-30,1.236923,20
48149,ZURN.S,2010-02-28,2010-03-31,0.622809,23
30553,ZURN.S,2010-01-31,2010-02-28,1.339416,20


,date,stock_RIC,index_member,stock_value_held,FUND_stock_value_held,market_cap,ETF_ownership,FUND_ownership,price,return1Mo,gross_profit,price_to_BV,DATE_monthly_std_dev,monthly_std_dev,count_returns
0,2009-12-31,0MW4EUR.xbo^K15,0,14.731695,9.666637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
1,2010-01-31,0MW4EUR.xbo^K15,0,11.226477,9.598608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
2,2010-02-28,0MW4EUR.xbo^K15,0,9.887296,8.259427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
3,2010-03-31,0MW4EUR.xbo^K15,0,11.585353,9.845493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
4,2010-04-30,0MW4EUR.xbo^K15,0,15.047999,9.688948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196219,2023-07-31,ZURN.S,1,5870.052977,3874.537377,6.606391e+10,0.088854,0.058648,439.079073,-0.964933,NaN,2.772687,2023-08-31,0.727499,22.0
196220,2023-08-31,ZURN.S,1,5816.278767,3834.866671,6.344198e+10,0.091679,0.060447,433.477862,-1.378327,NaN,2.907803,2023-09-30,0.588205,21.0
196221,2023-09-30,ZURN.S,1,5789.045875,3802.279416,6.357877e+10,0.091053,0.059804,434.412512,1.669894,NaN,2.841227,2023-10-31,0.910886,22.0
196222,2023-10-31,ZURN.S,1,6022.326667,3973.467134,6.545329e+10,0.092010,0.060707,447.220479,2.499405,NaN,2.926635,2023-11-30,0.619786,22.0


## Amihud ratio

In [94]:
import pandas as pd

stock_level_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/stock_level_data/stock_level_data.csv"
stock_level_df = pd.read_csv(stock_level_path)

panel_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv"
panel_df = pd.read_csv(panel_path)

#####################################################
stock_level_df['date'] = pd.to_datetime(stock_level_df['date'])
panel_df['date'] = pd.to_datetime(panel_df['date'])

# Calculate the Amihud ratio
stock_level_df['amihud_ratio'] = abs(stock_level_df['return1D']) / stock_level_df['volume']

# Group by stock and month, and calculate the mean Amihud ratio for each month
stock_level_df['month'] = stock_level_df['date'].dt.to_period('M')
monthly_amihud = stock_level_df.groupby(['stock_RIC', 'month']).agg({
    'amihud_ratio': 'mean',
    'date': 'max'  # Takes the last available date in the month, which might not be the last day
}).reset_index()

# Set each date to the last day of the respective month
monthly_amihud['date'] = monthly_amihud['month'].dt.to_timestamp(how='end').dt.date

# Drop the 'month' column as it's no longer needed
monthly_amihud.drop('month', axis=1, inplace=True)
monthly_amihud['date'] = pd.to_datetime(monthly_amihud['date'])

######### merge this shit
merged_df = pd.merge(panel_df, monthly_amihud, on=['date', 'stock_RIC'], how='left')
display(merged_df)

# Save the merged DataFrame
merged_df.to_csv(panel_path, index = False)

,date,stock_RIC,index_member,stock_value_held,FUND_stock_value_held,market_cap,ETF_ownership,FUND_ownership,price,return1Mo,gross_profit,price_to_BV,DATE_monthly_std_dev,monthly_std_dev,count_returns,amihud_ratio
0,2009-12-31,0MW4EUR.xbo^K15,0,14.731695,9.666637,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2010-01-31,0MW4EUR.xbo^K15,0,11.226477,9.598608,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2010-02-28,0MW4EUR.xbo^K15,0,9.887296,8.259427,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2010-03-31,0MW4EUR.xbo^K15,0,11.585353,9.845493,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2010-04-30,0MW4EUR.xbo^K15,0,15.047999,9.688948,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
196219,2023-07-31,ZURN.S,1,5870.052977,3874.537377,6.606391e+10,0.088854,0.058648,439.079073,-0.964933,NaN,2.772687,2023-08-31,0.727499,22.0,0.000003
196220,2023-08-31,ZURN.S,1,5816.278767,3834.866671,6.344198e+10,0.091679,0.060447,433.477862,-1.378327,NaN,2.907803,2023-09-30,0.588205,21.0,0.000003
196221,2023-09-30,ZURN.S,1,5789.045875,3802.279416,6.357877e+10,0.091053,0.059804,434.412512,1.669894,NaN,2.841227,2023-10-31,0.910886,22.0,0.000002
196222,2023-10-31,ZURN.S,1,6022.326667,3973.467134,6.545329e+10,0.092010,0.060707,447.220479,2.499405,NaN,2.926635,2023-11-30,0.619786,22.0,0.000003


### past 12-month returns

In [3]:
import pandas as pd

panel_path = "/Users/jonathanzeh/Library/CloudStorage/OneDrive-Personal/BA_Thesis/BA_coding/datasets/eikon_data/variable_data/monthly_panel_v1.csv"
df = pd.read_csv(panel_path)

# Convert 'date' to datetime format and ensure it is sorted
df['date'] = pd.to_datetime(df['date'])
df.sort_values(by=['stock_RIC', 'date'], inplace=True)

# Calculate the price 12 months ago by shifting the prices within each stock group
df['price_12_months_ago'] = df.groupby('stock_RIC')['price'].shift(12)

# Calculate the 12-month cumulative return
df['cumulative_return_12m'] = (df['price'] - df['price_12_months_ago']) / df['price_12_months_ago']

# Drop the helper column if it's no longer needed
df.drop('price_12_months_ago', axis=1, inplace=True)

df.to_csv(panel_path)